# Set-Up

In [ ]:
import cv2
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import os
from pathlib import Path
import uuid
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def normalize_yolo_box_1(four_p_box, height, width):
  int_cords = []
  for p in four_p_box:
    int_cords.append(p[0])
    int_cords.append(p[1])

  x0, y0, x1, y1, x2, y2, x3, y3 = int_cords

  # aahhhware
  x0 = x0 / width
  y0 = y0 / height
  x1 = x1 / width
  y1 = y1 / height
  x2 = x2 / width
  y2 = y2 / height
  x3 = x3 / width
  y3 = y3 / height

  return x0, y0, x1, y1, x2, y2, x3, y3

In [ ]:
def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    xy_center = (rotation_mat[0, 2], rotation_mat[1, 2])

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat, rotation_mat

In [ ]:
def get_random_back_image(backgroung_images_path):
  back_path_list = list(backgroung_images_path.glob('*.*'))
  back_idx = np.random.randint(0, len(back_path_list))
  back_img = cv2.imread(str(back_path_list[back_idx]), -1)
  # back_img = cv2.cvtColor(back_img, cv2.COLOR_BGR2RGB)

  # back_img shape is always the same, 1000x1000
  # cuz yolo squeezes images to squares
  back_img = cv2.resize(back_img, (1000, 1000))

  return back_img

def get_random_receipt_image(receipts_images_path):
  receipts_path_list = list(receipts_images_path.glob('*.*'))
  receipt_idx = np.random.randint(0, len(receipts_path_list))
  receipt_img = cv2.imread(str(receipts_path_list[receipt_idx]), -1)

  return receipt_img

# Open CV Augmentation Cooking

In [ ]:
backgroung_images_path = Path('/content/drive/MyDrive/receipts_dataset/yolo_dataset/backgrounds')
receipts_images_path = Path('/content/drive/MyDrive/receipts_dataset/yolo_dataset/clear_images')
samples_saving_path = Path('/content/drive/MyDrive/receipts_dataset/yolo_dataset/gen_samples')

In [ ]:
# generate sample returns result image with 1 to 4 receipts
# and their yolo-formatted points

def good_generate_sample(backgroung_images_path, receipts_images_path):

  # get num of receipts
  receipts_to_gen = np.random.randint(1, 5)
  # print(f'receipts to gen {receipts_to_gen}')

  back_img = get_random_back_image(backgroung_images_path)

  # init anchor points with some offset
  cur_x, cur_y = 30, 30

  # receipt / back.width max scale
  max_of_background = 1.0 / receipts_to_gen - cur_x / back_img.shape[1]

  # list of yolo points for each receipt
  receipts_yolo_points = []

  for i in range(receipts_to_gen):
    receipt_img = get_random_receipt_image(receipts_images_path)

    original_four_points = [(0, 0), (receipt_img.shape[1], 0), (receipt_img.shape[1], receipt_img.shape[0]), (0, receipt_img.shape[0])]

    # rotating image
    angle = np.random.randint(-30, 30)
    receipt_img, rotation_mat = rotate_image(receipt_img, angle)
    # print(f'angle is {angle}')
    # print(f'rotated receipt shape {receipt_img.shape}')

    # calculating rotated points of the receipt
    receipt_rotated_points = []
    for point in original_four_points:
      # convert to homogenous coordinates in np array format first so that you can pre-multiply M
      rotated_point = rotation_mat.dot(np.array(point + (1,)))
      receipt_rotated_points.append(rotated_point)
    # print(f'receipt rotated points {receipt_rotated_points}')

    # resize receipt if needed AND ALSO SCALE ROTATED POINTS ACCORDINGLY
    if receipt_img.shape[1] > back_img.shape[1] * max_of_background:
      # const keeps resulting image max 'max_of_background' width of back image
      resizing_ratio = back_img.shape[1] * max_of_background / receipt_img.shape[1]
      receipt_img = cv2.resize(receipt_img, (0, 0), fx = resizing_ratio, fy = resizing_ratio)
      for point in receipt_rotated_points:
        point[0] *= resizing_ratio
        point[1] *= resizing_ratio

    if receipt_img.shape[0] > back_img.shape[0] * 0.95:
      resizing_ratio = back_img.shape[0] * 0.95 / receipt_img.shape[0]
      receipt_img = cv2.resize(receipt_img, (0, 0), fx = resizing_ratio, fy = resizing_ratio)
      for point in receipt_rotated_points:
        point[0] *= resizing_ratio
        point[1] *= resizing_ratio

    # print(f'receipt shape resized {receipt_img.shape}')

    # add offset of anchor points to rotated points
    for point in receipt_rotated_points:
        point[0] += cur_x
        point[1] += cur_y

    # calculate normalized points for yolo format
    yolo_norm_points = normalize_yolo_box_1(receipt_rotated_points, back_img.shape[0], back_img.shape[1])
    receipts_yolo_points.append(yolo_norm_points)
    # print(f'yolo points {yolo_norm_points}')


    # placing receipt on the background
    # area for placing png receipt on the backgound
    x1, x2 = cur_x, cur_x + receipt_img.shape[1]
    y1, y2 = cur_y, cur_y + receipt_img.shape[0]

    # print(f'x1 y1 x2 y2 {x1, y1, x2, y2}')

    # dirty way to take care of the alpha channel
    alpha_s = receipt_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    for c in range(0, 3):
      back_img[y1:y2, x1:x2, c] = (alpha_s * receipt_img[:, :, c] +
                                  alpha_l * back_img[y1:y2, x1:x2, c])

    # update anchor points
    if angle > 0:
      cur_x = int(receipt_rotated_points[2][0])
      cur_y = int(receipt_rotated_points[1][1])
    else:
      cur_x = int(receipt_rotated_points[1][0])
      cur_y = int(receipt_rotated_points[0][1])

    # print(f'anchor (x, y) {cur_x, cur_y}')

    # plotting for testing
    # for point in receipt_rotated_points:
    #   back_img = cv2.circle(back_img, (int(point[0]), int(point[1])),
    #                         radius=20, color=(100, 100, 255), thickness=-1)

    # back_img = cv2.circle(back_img, (cur_x, cur_y),
    #                       radius=20, color=(200, 100, 255), thickness=-1)

  # plt.figure()
  # plt.imshow(back_img)

  return back_img, receipts_yolo_points

res = good_generate_sample(backgroung_images_path, receipts_images_path)

# 100 Random Tests

In [ ]:
# run 100 test hehe
for i in range(100):
  good_generate_sample(backgroung_images_path, receipts_images_path)

# Generating Yolo Ds

In [ ]:
# working nice so it's time to generate yolo dataset

synt_yolo_ds_path = Path('/content/drive/MyDrive/receipts_dataset/synthetic_yolo_dataset')

def generate_yolo_item_from_sample(synt_yolo_ds_path, split_name):
  # chosing appropriate directory for image and labels
  images_path = os.path.join(synt_yolo_ds_path, 'images', split_name)
  labels_path = os.path.join(synt_yolo_ds_path, 'labels', split_name)

  # generate sample image, get receipt points
  sample_img, receipts_yolo_points = good_generate_sample(backgroung_images_path, receipts_images_path)

  # for each image generate {filename}.txt with all points and class_id 0
  generated_filename = uuid.uuid4().hex

  with open(os.path.join(labels_path, f'{generated_filename}.txt'), 'w') as f:
    for coords in receipts_yolo_points:
      f.write(f'0 ' + ' '.join(map(str, coords)) + '\n')

  # and save each sample image
  cv2.imwrite(os.path.join(images_path, f'{generated_filename}.jpg'), sample_img)


In [ ]:
# generate 900 training examples

train_samples_count = 900

for i in range(train_samples_count):
  generate_yolo_item_from_sample(synt_yolo_ds_path, split_name='train')

In [ ]:
# generate 100 validation examples

val_samples_count = 100

for i in range(val_samples_count):
  generate_yolo_item_from_sample(synt_yolo_ds_path, split_name='val')